In [1]:
%env KREAS_BACKEND=tensorflow

env: KREAS_BACKEND=tensorflow


In [2]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

In [3]:
from keras.datasets import imdb

In [4]:
(x_train, y_train),(x_test, y_test) = imdb.load_data(num_words=10000)

<__array_function__ internals>:5: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
/Users/hao/opt/anaconda3/lib/python3.8/site-packages/tensorflow/python/keras/datasets/imdb.py:159: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  x_train, y_train = np.array(xs[:idx]), np.array(labels[:idx])
/Users/hao/opt/anaconda3/lib/python3.8/site-packages/tensorflow/python/keras/datasets/imdb.py:160: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is depr

In [5]:
len(x_train)

25000

In [6]:
x_train[0] ##每個數字都代表一個字（編碼） 最常出現就是1號其次2號，以此類推

[1,
 14,
 22,
 16,
 43,
 530,
 973,
 1622,
 1385,
 65,
 458,
 4468,
 66,
 3941,
 4,
 173,
 36,
 256,
 5,
 25,
 100,
 43,
 838,
 112,
 50,
 670,
 2,
 9,
 35,
 480,
 284,
 5,
 150,
 4,
 172,
 112,
 167,
 2,
 336,
 385,
 39,
 4,
 172,
 4536,
 1111,
 17,
 546,
 38,
 13,
 447,
 4,
 192,
 50,
 16,
 6,
 147,
 2025,
 19,
 14,
 22,
 4,
 1920,
 4613,
 469,
 4,
 22,
 71,
 87,
 12,
 16,
 43,
 530,
 38,
 76,
 15,
 13,
 1247,
 4,
 22,
 17,
 515,
 17,
 12,
 16,
 626,
 18,
 2,
 5,
 62,
 386,
 12,
 8,
 316,
 8,
 106,
 5,
 4,
 2223,
 5244,
 16,
 480,
 66,
 3785,
 33,
 4,
 130,
 12,
 16,
 38,
 619,
 5,
 25,
 124,
 51,
 36,
 135,
 48,
 25,
 1415,
 33,
 6,
 22,
 12,
 215,
 28,
 77,
 52,
 5,
 14,
 407,
 16,
 82,
 2,
 8,
 4,
 107,
 117,
 5952,
 15,
 256,
 4,
 2,
 7,
 3766,
 5,
 723,
 36,
 71,
 43,
 530,
 476,
 26,
 400,
 317,
 46,
 7,
 4,
 2,
 1029,
 13,
 104,
 88,
 4,
 381,
 15,
 297,
 98,
 32,
 2071,
 56,
 26,
 141,
 6,
 194,
 7486,
 18,
 4,
 226,
 22,
 21,
 134,
 476,
 26,
 480,
 5,
 144,
 30,
 5535,
 18,

In [7]:
for i in range(10):
    print(len(x_train[i]),end =', ')

218, 189, 141, 550, 147, 43, 123, 562, 233, 130, 

In [8]:
y_train[:10] ##0負評，1正評

array([1, 0, 0, 1, 0, 0, 1, 0, 1, 0])

### 送入神經網路的輸入處理

雖然 RNN 是可以處理不同長度的輸入, 在寫程式時我們還是要

* 設輸入文字長度的上限
* 把每段文字都弄成一樣長, 太短的後面補上 0

In [9]:
from keras.preprocessing import sequence ##做資料串列處理 

In [10]:
x_train = sequence.pad_sequences(x_train, maxlen = 100)
x_teat = sequence.pad_sequences(x_test, maxlen=100)

In [11]:
x_train.shape

(25000, 100)

### 決定神經網路架構

* 先將 10000 維的文字壓到 128 維
* 然後用 128 個 LSTM
* 最後一個 output, 直接用 sigmoid 送出

文字我們用 1-hot 表示是很標準的方式, 不過要注意的是, 因為我們指定要 1 萬個字, 所以每個字是用 1 萬維的向量表示! 這一來很浪費記憶空間, 二來字和字間基本上是沒有關係的。我們可以用某種「合理」的方式, 把字壓到比較小的維度, 這些向量又代表某些意思 (比如說兩個字代表的向量角度小表相關程度大) 等等。

這聽來很複雜的事叫 "word embedding", 而事實上 Keras 會幫我們做。我們只需告訴 Keras 原來最大的數字是多少 (10000), 還有我們打算壓到幾維 (128)。

In [12]:
from keras.models import Sequential
from keras.layers import Dense,Embedding
from keras.layers import LSTM

In [13]:
model = Sequential()

In [14]:
model.add(Embedding(10000,128))

In [15]:
model.add(LSTM(150))

In [16]:
model.add(Dense(1, activation='sigmoid'))

In [17]:
model.summary() 

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 128)         1280000   
_________________________________________________________________
lstm (LSTM)                  (None, 150)               167400    
_________________________________________________________________
dense (Dense)                (None, 1)                 151       
Total params: 1,447,551
Trainable params: 1,447,551
Non-trainable params: 0
_________________________________________________________________


In [18]:
model.compile(loss='binary_crossentropy',
             optimizer='adam',
             metrics=['accuracy'])

In [26]:
model.fit(x_train, y_train,
          batch_size=32,
          epochs=10)

Epoch 1/10
782/782 [==============================] - 79s 101ms/step - loss: 0.1851 - accuracy: 0.9281
Epoch 2/10
782/782 [==============================] - 89s 114ms/step - loss: 0.1294 - accuracy: 0.9530
Epoch 3/10
782/782 [==============================] - 87s 112ms/step - loss: 0.0904 - accuracy: 0.9681
Epoch 4/10
782/782 [==============================] - 87s 111ms/step - loss: 0.0700 - accuracy: 0.9766
Epoch 5/10
782/782 [==============================] - 86s 110ms/step - loss: 0.0450 - accuracy: 0.9852
Epoch 6/10
782/782 [==============================] - 82s 105ms/step - loss: 0.0447 - accuracy: 0.9850
Epoch 7/10
782/782 [==============================] - 81s 103ms/step - loss: 0.0261 - accuracy: 0.9913
Epoch 8/10
782/782 [==============================] - 81s 103ms/step - loss: 0.0266 - accuracy: 0.9917
Epoch 9/10
782/782 [==============================] - 82s 105ms/step - loss: 0.0262 - accuracy: 0.9919
Epoch 10/10
782/782 [==============================] - 80s 102ms/step - l

In [31]:
score = model.evaluate(x_teat, y_test)

782/782 [==============================] - 23s 29ms/step - loss: 0.9262 - accuracy: 0.8289


In [32]:
print('測試資料的 loss', score[0])
print('測試資料的正確率', score[1])

測試資料的 loss 0.9262150526046753
測試資料的正確率 0.8288800120353699


In [35]:
model_json = model.to_json()
open('imdb_model.json','w').write(model_json)

1965

In [34]:
model.save_weights('imdb_model.h5')